In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model, models, mixed_precision
from tensorflow.keras.utils import plot_model
import tensorflow_text
import tensorflow_hub as hub
import tensorflow_addons as tfa
import glob

physical_devices = tf.config.experimental.list_physical_devices('GPU')
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
mixed_precision.set_global_policy('mixed_float16')


# all_files = glob.glob("*MA.csv")
# li = []
# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)
#
# df = pd.concat(li, axis=0, ignore_index=True)
# df.dropna(inplace=True)
# df = df[['chapter_name', 'qns']]
# df.to_csv('all_qns.csv', index=False)

df = pd.read_csv('all_qns_multi_label.csv')
labels = df['chapter_name'].str.lower().tolist()
labels = [[*i.split(',')] for i in labels]
classes = []
for i in labels:
    for j in i:
        if j not in classes:
            classes.append(j)
classes_index = {v: i for i, v in enumerate(classes)}
def multi_hot_encode(label):
    label_encoded = [classes_index[x] for x in label]
    label_encoded = tf.reduce_max(tf.one_hot(label_encoded, len(classes)), axis=0)
    return label_encoded

labeled_df = pd.DataFrame()
labels = [multi_hot_encode(i) for i in labels]

def clean(qns):
    import re
    qns = re.sub(r'[^\x00-\x7F]+', ' ', qns)  # clean unicode stuff
    qns = re.sub(r'\d+', ' 0 ', qns)  # replace all numbers with 0
    qns = re.sub(r'_+', '_', qns)  # replace all underscores with single underscore
    qns = qns.split('(Note to students')[0].split('Notes to student')[0].split('Note to students')[0].split('Note to student')[0].split('Notes to students')[0].split('(Separate ')[0]
    qns = qns.strip()
    return qns


labeled_df['label'] = pd.Series(labels)
labeled_df['qns'] = df['qns'].apply(clean)
labeled_df.dropna(inplace=True)
labeled_df.to_csv('multi_labeled_df.csv', index=False)
labeled_df = labeled_df.sample(frac=1).reset_index(drop=True)  # shuffle
labels

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3080 Ti, compute capability 8.6


C:\Program Files\Python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0-rc0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


[<tf.Tensor: shape=(36,), dtype=float32, numpy=
 array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], dtype=float32)>,
 <tf.Tensor: shape=(36,), dtype=float32, numpy=
 array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], dtype=float32)>,
 <tf.Tensor: shape=(36,), dtype=float32, numpy=
 array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], dtype=float32)>,
 <tf.Tensor: shape=(36,), dtype=float32, numpy=
 array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], dtype=float32)>,
 <tf.Tensor: shape=(36,), dtype=float32, numpy=
 array([1., 1., 

In [2]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
batch_size = 128
epochs = 100
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=15, verbose=1,
                                     mode='auto', baseline=None, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=12, verbose=1)
]

In [3]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessing_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer('https://tfhub.dev/google/experts/bert/wiki_books/qnli/2', trainable=True)
outputs = encoder(encoder_inputs)
x = outputs['pooled_output']
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.2)(x)
xOut = Dense(len(classes), activation=None)(x)
model = Model(text_input, xOut)
model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])

In [4]:
plot_model(model, show_dtype=True, show_shapes=True, show_layer_names=True, to_file='model.png')
model.summary()
model.fit(np.array(labeled_df['qns']), np.asarray(labels), batch_size=batch_size, epochs=epochs, callbacks=callbacks, verbose=1, use_multiprocessing=True, validation_split=0.2)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['input_1[0][0]']                
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

C:\Program Files\Python39\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/StatefulPartitionedCall:1", shape=(None,), dtype=int32), values=Tensor("mul_2:0", dtype=float32), dense_shape=Tensor("Adam/gradients/StatefulPartitionedCall:2", shape=(None,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


191/191 [==============================] - 69s 185ms/step - loss: 0.2858 - accuracy: 0.1108 - val_loss: 0.3052 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/100
191/191 [==============================] - 49s 258ms/step - loss: 0.2746 - accuracy: 0.1410 - val_loss: 0.3038 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/100
191/191 [==============================] - 27s 141ms/step - loss: 0.2733 - accuracy: 0.1523 - val_loss: 0.3057 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/100
191/191 [==============================] - 27s 140ms/step - loss: 0.2724 - accuracy: 0.1576 - val_loss: 0.3033 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 5/100
191/191 [==============================] - 27s 141ms/step - loss: 0.2720 - accuracy: 0.1595 - val_loss: 0.3047 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 6/100
 34/191 [====>.........................] - ETA: 20s - loss: 0.2712 - accuracy: 0.1654

KeyboardInterrupt: 

In [ ]:
model.save('model.h5')
question = 'The height of an isosceles triangle with a base length of 8cm is 3cm. What is the perimeter of a similar triangle with base 4cm'
sigmoid_out = model(tf.constant([question])).numpy()[0]
confidence = [sigmoid_out.index(i) for i in sigmoid_out if i > 0.5]
class_predicted = [classes[confidence.index(i)] for i in confidence]
for class_, conf in zip(class_predicted, confidence):
    print(f'Class predicted: {class_} with confidence {conf*100:.3f}%')
conf_list = {}
for i in range(len(classes)):
    conf_list.update({classes[i]: f'{sigmoid_out[i]*100:.3f}%'})

conf_list